# Intialize notebook

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
%matplotlib inline

from tqdm import tqdm

import sys
sys.path.insert(1, './python_files/yolo_load_files')
from predictor import get_estimations


In [2]:
# Constants
#KITTI_DATASET = '/media/javier/HDD_linux/KITTI_dataset/training/'
KITTI_DATASET = '/media/robesafe/SSD_SATA/KITTI_DATASET/'
WORK_PATH = os.getcwd()
WEIGHTS_PATH = os.getcwd()+'/weights'

In [4]:
WEIGHTS_PATH

'/home/robesafe/Javier/3D-detection-system-lidar-camera/weights'

In [8]:
model_lst = [x for x in sorted(os.listdir(WEIGHTS_PATH)) if x.endswith('.pkl')]
print(model_lst)

['KITTI_100_YOLO5l.pkl']


# Run YOLOv5

In [3]:
yolo_estimations = get_estimations(KITTI_DATASET, WORK_PATH, WEIGHTS_PATH, batch_size=400, threshold=0.5)

Using cache found in /home/robesafe/Javier/3D-detection-system-lidar-camera/ultralytics_yolov5_master
YOLOv5 🚀 71cddd1 torch 1.10.2 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11175.375MB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


Using previous model KITTI_100_YOLO5l.pkl


Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /home/robesafe/Javier/3D-detection-system-lidar-camera/checkpoints/vgg19_bn-c79401a0.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [4]:
yolo_estimations

[[[(711, 149), (804, 303), 'person', 0.93115234375],
  [(371, 206), (420, 274), 'bicycle', 0.640625],
  [(337, 207), (382, 280), 'bicycle', 0.59619140625],
  [(911, 195), (962, 277), 'bicycle', 0.56005859375],
  [(250, 212), (330, 283), 'bicycle', 0.50341796875]],
 [[(388, 179), (422, 198), 'car', 0.7431640625]],
 [[(658, 189), (702, 221), 'car', 0.56591796875]],
 [[(614, 184), (722, 282), 'car', 0.8857421875],
  [(0, 232), (210, 371), 'car', 0.80078125]],
 [[(281, 184), (342, 211), 'car', 0.8466796875],
  [(366, 181), (403, 203), 'car', 0.77197265625],
  [(403, 171), (436, 191), 'car', 0.54443359375]],
 [[(329, 179), (356, 235), 'person', 0.75634765625]],
 [[(40, 185), (220, 242), 'car', 0.8603515625],
  [(509, 171), (574, 210), 'car', 0.76708984375],
  [(331, 171), (395, 204), 'car', 0.6591796875],
  [(580, 168), (603, 187), 'car', 0.50537109375]],
 [[(564, 174), (613, 223), 'car', 0.767578125],
  [(540, 175), (566, 192), 'car', 0.6962890625],
  [(480, 180), (511, 200), 'car', 0.6865

In [5]:
yolo_detecctions = []
n_image = 0
for image in yolo_estimations:
    n_detecction = 0
    for detecction in image:
        obj_type = None
        if detecction[2] == "car":
            obj_type = "Car"
        elif detecction[2] == "person":
            obj_type = "Pedestrian"
        elif detecction[2] == "bicycle":
            obj_type = "Cyclist"
        else:
            continue
        yolo_detecctions.append([n_image, n_detecction, obj_type, detecction[0][0], detecction[0][1],
                                 detecction[1][0], detecction[1][1], detecction[3]])
        n_detecction += 1
    n_image += 1

In [6]:
df_detections = pd.DataFrame(columns=['frame','id','type','left','top','right','bottom','score'], data=yolo_detecctions)
df_detections

,frame,id,type,left,top,right,bottom,score
0,0,0,Pedestrian,711,149,804,303,0.931152
1,0,1,Cyclist,371,206,420,274,0.640625
2,0,2,Cyclist,337,207,382,280,0.596191
3,0,3,Cyclist,911,195,962,277,0.560059
4,0,4,Cyclist,250,212,330,283,0.503418
...,...,...,...,...,...,...,...,...
36367,7480,9,Car,611,175,643,199,0.622559
36368,7480,10,Car,436,180,494,212,0.588379
36369,7480,11,Car,518,175,545,193,0.555176
36370,7480,12,Car,355,179,468,224,0.542969


In [7]:
def get_bounding_boxes_2D(df, image_id):
    df_frame = df[df['frame'] == image_id]
    bbs = []
    for index, row in df_frame.iterrows():
        bb = (row['left'], row['top'], row['right'], row['bottom'], row['type'])
        bbs.append(bb)
    return bbs

def show_bounding_boxes_2D(df, image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    img = KITTI_DATASET+'images/'+name+'.png'
    
    # do projection staff
    plt.figure(figsize=(12,5),dpi=96,tight_layout=True)
    png = mpimg.imread(img)
    IMG_H,IMG_W,_ = png.shape
    # restrict canvas in range
    plt.axis([0,IMG_W,IMG_H,0])
    plt.imshow(png)
    
    # draw bounding boxes
    bbs = get_bounding_boxes_2D(df, image_id)
    for bb in bbs:
        if bb[4] == 'Car':
            color = 'red'
        elif bb[4] == 'Pedestrian':
            color = 'green'
        elif bb[4] == 'Cyclist':
            color = 'blue'
        else:
            color = 'yellow'
        plt.gca().add_patch(Rectangle((bb[0], bb[1]),bb[2]-bb[0],bb[3]-bb[1],linewidth=1,edgecolor=color,facecolor='none'))
    
    # plt.savefig(WORK_PATH+'/yolo_images/{}.png'.format(name),bbox_inches='tight')
    plt.show()

In [8]:
i = 0

In [9]:
show_bounding_boxes_2D(df_detections, i)
i += 1

/home/robesafe/anaconda3/envs/javi_tfm/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


# Associate GT with YOLOv5 detections

In [10]:
df_gt = pd.read_csv(KITTI_DATASET+'kitti_gt.csv')

In [28]:
df_gt

,frame,id,type,truncated,occluded,alpha,left,top,right,bottom,height,width,length,x,y,z,ry,distance,heigth_image,width_image,theta_ray,incomplete_2d_horizontal,incomplete_2d_bottom
0,0,0,Pedestrian,0.0,0.0,-0.20,712.40,143.00,810.73,307.92,1.89,0.48,1.20,1.84,1.47,8.41,0.01,8.733533,164.92,98.33,0.21,False,False
1,1,0,Car,0.0,0.0,1.85,387.63,181.54,423.81,203.12,1.67,1.87,3.69,-16.53,2.39,58.49,1.57,60.827897,21.58,36.18,-0.28,False,False
2,1,1,Cyclist,0.0,3.0,-1.65,676.60,163.95,688.98,193.93,1.86,0.60,2.02,4.59,1.32,45.84,-1.55,46.088134,29.98,12.38,0.10,False,False
3,2,0,Car,0.0,0.0,-1.67,657.39,190.13,700.07,223.39,1.41,1.58,4.36,3.18,2.27,34.38,-1.58,34.601296,33.26,42.68,0.09,False,False
4,3,0,Car,0.0,0.0,1.55,614.24,181.78,727.31,284.77,1.57,1.73,4.15,1.00,1.75,13.22,1.62,13.372767,102.99,113.07,0.07,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34851,7480,9,Car,0.0,2.0,-0.33,749.78,170.88,917.51,243.48,1.77,1.69,3.82,5.72,1.75,18.85,-0.04,19.776334,72.60,167.73,0.29,False,False
34852,7480,10,Car,0.0,1.0,-2.90,344.07,180.05,479.10,224.15,1.49,1.59,4.44,-7.12,1.78,26.03,3.12,27.044846,44.10,135.03,-0.26,False,False
34853,7480,11,Car,0.0,2.0,-2.91,382.70,179.71,495.48,219.10,1.48,1.63,4.10,-6.79,1.78,28.78,-3.14,29.623654,39.39,112.78,-0.23,False,False
34854,7480,12,Car,0.0,0.0,-1.69,631.71,179.85,678.40,214.37,1.41,1.56,4.08,1.90,1.74,31.98,-1.63,32.083610,34.52,46.69,0.06,False,False


In [29]:
def bbox_2d_overlap(bbox_gt, bbox_det):
    """
    Calculate the overlap between two bounding boxes in 2D.

    :param bbox_gt: (xmin, ymin, xmax, ymax)
    :param bbox_det: (xmin, ymin, xmax, ymax)
    :return: overlap: float
    """
    # Calculate the area of the bounding boxes
    area_gt = (bbox_gt[2] - bbox_gt[0]) * (bbox_gt[3] - bbox_gt[1])
    area_det = (bbox_det[2] - bbox_det[0]) * (bbox_det[3] - bbox_det[1])
    # Calculate the intersection of the areas
    intersection = bbox_2d_intersection(bbox_gt, bbox_det)
    # Calculate the union of the areas
    union = area_gt + area_det - intersection
    # Calculate the overlap
    overlap = intersection / union
    #print(overlap)
    #print("#################")
    return overlap

def bbox_2d_intersection(bbox_gt, bbox_det):
    """
    Calculate the intersection of two bounding boxes in 2D.

    :param bbox_gt: (xmin, ymin, xmax, ymax)
    :param bbox_det: (xmin, ymin, xmax, ymax)
    :return: intersection: float
    """
    # Calculate the intersection
    intersection = max(0, min(bbox_gt[2], bbox_det[2]) - max(bbox_gt[0], bbox_det[0])) * \
                   max(0, min(bbox_gt[3], bbox_det[3]) - max(bbox_gt[1], bbox_det[1]))
    #print(bbox_gt)
    #print(bbox_det)
    #print(intersection)
    return intersection

In [32]:
combined_list = []

# Get different values of column 'frame' and sort them
frames = df_gt['frame'].unique()
frames.sort()

# Create loading bar
loading_bar = tqdm(total=len(frames))

# Iterate over frames
for frame in frames:
    # Iterate over the df_gt rows with the same 'frame' value
    for index_gt, row_gt in df_gt[df_gt['frame'] == frame].iterrows():
        best_overlap = None
        # Iterate over the df_detections rows with the same 'frame' value and the same 'type' value
        for index_det, row_det in df_detections[(df_detections['frame'] == frame) &
                                                (df_detections['type'] == row_gt['type'])].iterrows():
            # If row_gt['type'] == 'Car'
            if row_gt['type'] == 'Car':
                # If the bounding boxes 2D have an overlap greater than 0.7
                if bbox_2d_overlap((row_gt['left'], row_gt['top'], row_gt['right'], row_gt['bottom']),
                                   (row_det['left'], row_det['top'], row_det['right'], row_det['bottom'])) > 0.7:
                    # Store the best overlaping bounding box
                    best_overlap = row_det
            # If row_gt['type'] == 'Pedestrian' or row_gt['type'] == 'Cyclist'
            else:
                # If the bounding boxes 2D have an overlap greater than 0.5
                if bbox_2d_overlap((row_gt['left'], row_gt['top'], row_gt['right'], row_gt['bottom']),
                                   (row_det['left'], row_det['top'], row_det['right'], row_det['bottom'])) > 0.5:
                    # Store the best overlaping bounding box
                    best_overlap = row_det
        # If best_overlap is None
        if best_overlap is None:
            continue
        # If best_overlap is not None
        else:
            combined_list.append([row_gt['frame'], row_gt['id'], row_gt['type'], row_gt['left'], row_gt['top'],
                                  row_gt['right'], row_gt['bottom'], row_gt['distance'], best_overlap['type'],
                                  best_overlap['left'], best_overlap['top'], best_overlap['right'],
                                  best_overlap['bottom'], best_overlap['score']])
    # Update loading bar
    loading_bar.update(1)

100%|█████████▉| 7480/7481 [01:23<00:00, 96.67it/s] 

In [34]:
df_combined = pd.DataFrame(columns=['frame','id','type_gt','left_gt','top_gt','right_gt','bottom_gt','distance_gt',
                                    'type_yolo','left_yolo','top_yolo','right_yolo','bottom_yolo','score_yolo'],
                           data=combined_list)

In [35]:
df_combined

,frame,id,type_gt,left_gt,top_gt,right_gt,bottom_gt,distance_gt,type_yolo,left_yolo,top_yolo,right_yolo,bottom_yolo,score_yolo
0,0,0,Pedestrian,712.40,143.00,810.73,307.92,8.733533,Pedestrian,711,149,804,303,0.931152
1,2,0,Car,657.39,190.13,700.07,223.39,34.601296,Car,658,189,702,221,0.565918
2,3,0,Car,614.24,181.78,727.31,284.77,13.372767,Car,614,184,722,282,0.885742
3,4,0,Car,280.38,185.10,344.90,215.59,41.416148,Car,281,184,342,211,0.846680
4,5,0,Pedestrian,330.06,178.74,360.77,238.64,24.626313,Pedestrian,329,179,356,235,0.756348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19839,7480,0,Car,0.00,185.93,214.05,348.86,10.618070,Car,0,187,209,327,0.926270
19840,7480,4,Car,1094.88,190.09,1241.00,374.00,8.439579,Car,1102,203,1242,369,0.920898
19841,7480,5,Car,934.82,184.38,1241.00,326.25,10.388547,Car,939,190,1179,330,0.922852
19842,7480,10,Car,344.07,180.05,479.10,224.15,27.044846,Car,355,179,468,224,0.542969


100%|██████████| 7481/7481 [01:39<00:00, 96.67it/s]

# Comparing distace methods using YOLOv5 detections